In [ ]:
# Import dependencies
import pandas as pd
import random as random
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import linregress
from citipy import citipy
import requests
import json


import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [ ]:
# Kristina's code starts here

In [ ]:
# Kristina's code ends here

In [ ]:
# Jeff's code starts here
temp_data_df = pd.read_csv('Resources/clean_temp_data.csv')
temp_data_df


In [ ]:
# Jeff's code ends here
temp_data_df["Name"].unique()
temp_data_df.dropna(inplace = True)
temp_data_df

#code below combines all locations into one average value for month and year
list = temp_data_df["Name"].unique()
temp_comb_df = temp_data_df.groupby(["Date"])["Total Monthly Precipitation", "Total Montly Snowfall (mm)",
                                             "Avg Monthly Temp", "Monthly Mean Max Temp",
                                              "Monthly Mean Min Temp"].mean().reset_index()

#Data is string Year-Month (month two digits 01-12).  This splits into two fields
temp_comb_df["Month"] = temp_comb_df["Date"].str.split("-").str[1].astype(int)
temp_comb_df["Year"] = temp_comb_df["Date"].str.split("-").str[0].astype(int)

col_list = temp_comb_df.columns.tolist()
Season_data_df = pd.DataFrame(columns = col_list)

#combining months to form seasons: Winter (Dec-Feb), Fall(Mar-May), Summer(June-Aug), Fall(Sept-Nov)
for i in temp_comb_df["Year"].unique():
    if i!= 1950 and i!= 2020: #ignore 1950 because don't have dec 1949 and don't have all 2020
        
        #in dataframe the "month" represents the season: 2=winter, 4 = spring, 7 = summer, 10=fall
        winter_data = temp_comb_df.loc[((temp_comb_df["Year"]==i) & (temp_comb_df["Month"].between(1,2)))|\
                                         ((temp_comb_df["Year"]==(i-1)) & (temp_comb_df["Month"]==12))].mean()
        winter_data["Year"]=winter_data["Year"].astype(int)+1 # extra 1 because rounds down to year previous
        winter_data["Month"] = 1 #would be 5 otherwise (mean(12,1,2))
        spring_data = temp_comb_df.loc[((temp_comb_df["Year"]==i) & (temp_comb_df["Month"].between(3,5)))].mean()
        spring_data["Year"]=spring_data["Year"].astype(int)
        summer_data = temp_comb_df.loc[((temp_comb_df["Year"]==i) & (temp_comb_df["Month"].between(6,8)))].mean()
        summer_data["Year"]=summer_data["Year"].astype(int)
        fall_data = temp_comb_df.loc[((temp_comb_df["Year"]==i) & (temp_comb_df["Month"].between(9,11)))].mean() 
        fall_data["Year"]=fall_data["Year"].astype(int)
        
        frames = [winter_data, spring_data, summer_data, fall_data]
        Season_data_df = Season_data_df.append(frames, ignore_index = True)
        
Season_data_df["Temp Swing"] = Season_data_df["Monthly Mean Max Temp"]-Season_data_df["Monthly Mean Min Temp"]
Season_data_df.drop(columns=["Date"], inplace = True)


In [ ]:
Season_data_df

In [ ]:
variable = "Temp Swing" #pick variable to look at

#defining seasons by value in month column
winter = 1
spring = 4
summer = 7
fall = 10

#specifying data for plotting based on variable and season
x_axis = Season_data_df[Season_data_df["Month"]==winter]["Year"]
y_axis = Season_data_df[Season_data_df["Month"]==winter][variable]
plt.scatter(x_axis,y_axis)

(slope_N, intercept_N, rvalue_N, pvalue_N, stderr_N) = linregress(x_axis,y_axis)
regress_values_N = x_axis * slope_N + intercept_N
line_eq = "y = " + str(round(slope_N,2)) + "x + " + str(round(intercept_N,2))+", r = " +str(round(rvalue_N,3))\
                + "p ="+str(round(pvalue_N,3))
max_y = y_axis.max()
min_x = x_axis.min()
plt.plot(x_axis,regress_values_N,"r-")
plt.annotate(line_eq,(min_x+1,max_y-1),fontsize=11,color="red") #writes to coordinates on graph
plt.xlabel("Year")
plt.ylabel(f"{variable} ")
plt.title(f"{variable} versus Latitude /n for Northern Hemisphere")

In [ ]:
# Faduma's code starts here

In [ ]:
# Faduma's code ends here

In [ ]:
# Emily's code starts here

In [ ]:
storm_data = pd.read_csv('Resources/clean_storm_event_data.csv')
storm_data

In [ ]:
storm_data['Event Type'].unique()

# Counts of Tornado Eventsin Minnesota from 1950 to 2020

In [ ]:
year_list = storm_data["Year"].unique()
event_list = storm_data["Event Type"].unique()


clean_tornado_df = pd.DataFrame()
clean_tornado_df = storm_data[storm_data["Event Type"]=="Tornado"]


#clean_storm_data_mn_df
year_count_df = pd.DataFrame()
for i in year_list:
    year_count = clean_tornado_df[clean_tornado_df["Year"]==i]["Event Type"].count()
    year_count_df = year_count_df.append({"Year":i, "Count":year_count}, ignore_index=True)
year_count_df

x_data = (year_count_df["Year"].astype(int))
y_data = year_count_df["Count"].astype(float)

plt.scatter(x_data,y_data, s = 4)
plt.xlabel("Year")
plt.ylabel("Count")
plt.title("Counts of Tornado Events vs Year")

(slope_N, intercept_N, rvalue_N, pvalue_N, stderr_N) = linregress(x_data,y_data)
regress_values_N = x_data * slope_N + intercept_N
line_eq = "y = " + str(round(slope_N,2)) + "x + " + str(round(intercept_N,2))+", r = " +str(round(rvalue_N,2))+", p="+str(round(pvalue_N,2))
min_x = x_data.min()
max_y = y_data.max()
plt.plot(x_data,regress_values_N,"r-")
plt.annotate(line_eq,(min_x+1,max_y-1),fontsize=11,color="red") #writes to coordinates on graph
plt.show()

# Counts of Hail Events in Minnesota from 1950 to 2020

In [ ]:
clean_hail_df = pd.DataFrame()
clean_hail_df = storm_data[storm_data["Event Type"]=="Hail"]


#clean_storm_data_mn_df
year_count_df = pd.DataFrame()
for i in year_list:
    year_count = clean_hail_df[clean_hail_df["Year"]==i]["Event Type"].count()
    year_count_df = year_count_df.append({"Year":i, "Count":year_count}, ignore_index=True)
year_count_df

x_data = (year_count_df["Year"].astype(int))
y_data = year_count_df["Count"].astype(float)

plt.scatter(x_data,y_data, s = 4)
plt.xlabel("Year")
plt.ylabel("Count")
plt.title("Counts of Hail Events vs Year")

(slope_N, intercept_N, rvalue_N, pvalue_N, stderr_N) = linregress(x_data,y_data)
regress_values_N = x_data * slope_N + intercept_N
line_eq = "y = " + str(round(slope_N,2)) + "x + " + str(round(intercept_N,2))+", r = " +str(round(rvalue_N,2))+", p="+str(round(pvalue_N,2))
min_x = x_data.min()
max_y = y_data.max()
plt.plot(x_data,regress_values_N,"r-")
plt.annotate(line_eq,(min_x+1,max_y-1),fontsize=11,color="red") #writes to coordinates on graph
plt.show()

# Counts of Thunderstorm Wind Events in Minnesota from 1950 to 2020

In [ ]:
clean_thunderstorm_wind_df = pd.DataFrame()
clean_thunderstorm_wind_df = storm_data[storm_data["Event Type"]=="Thunderstorm Wind"]


#clean_storm_data_mn_df
year_count_df = pd.DataFrame()
for i in year_list:
    year_count = clean_thunderstorm_wind_df[clean_thunderstorm_wind_df["Year"]==i]["Event Type"].count()
    year_count_df = year_count_df.append({"Year":i, "Count":year_count}, ignore_index=True)
year_count_df

x_data = (year_count_df["Year"].astype(int))
y_data = year_count_df["Count"].astype(float)

plt.scatter(x_data,y_data, s = 4)
plt.xlabel("Year")
plt.ylabel("Count")
plt.title("Counts of Thunderstorm/Wind Events vs Year")

(slope_N, intercept_N, rvalue_N, pvalue_N, stderr_N) = linregress(x_data,y_data)
regress_values_N = x_data * slope_N + intercept_N
line_eq = "y = " + str(round(slope_N,2)) + "x + " + str(round(intercept_N,2))+", r = " +str(round(rvalue_N,2))+", p="+str(round(pvalue_N,2))
min_x = x_data.min()
max_y = y_data.max()
plt.plot(x_data,regress_values_N,"r-")
plt.annotate(line_eq,(min_x+1,max_y-1),fontsize=11,color="red") #writes to coordinates on graph
plt.show()

# Total Count of Tornado, Thunderstorm/Wind, and Hail Events in Minnesota from 1950 to 2020

In [ ]:
# Emily's code ends here